<a href="https://colab.research.google.com/github/siliconflow/siliconcloud-cookbook/blob/main/examples/how-to-handle-rate-limit-in-siliconcloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 如何处理速率限制
   当您反复调用SiliconCloud API时，可能会遇到429: 'Too Many Requests'或RateLimitError的错误信息，这些错误是由于超过API的速率限制引起的。

   本文提供了避免和处理速率限制错误的技巧。


## 速率限制的原因

速率限制是 API 的常见做法，它们的实施有几个不同的原因：
- 保障资源的公平性及合理利用：确保资源公平使用。 防止某些用户过多请求，影响其他用户的正常使用体验。
- 有效防止请求过载：帮助管理总负载情况，避免请求激增导致的服务器性能问题，提高服务可靠性。
- 安全防护：防止恶意性攻击，导致平台过载甚至服务中断。


### 速率限制衡量方式
平台速率限制目前以两种方式衡量：RPM（ requests per minute ，每分钟请求次数）、 TPM（ tokens per minute ，每分钟tokens ） 。

速率限制可能会因在任一选项（RPM、TPM）中达峰而触发，取决于哪个先发生。例如，在RPM限制为20，TPM限制为200K时，一分钟内，你向 ChatCompletions 发送了 20 个请求，每个请求只有 100 个 Token ，那么你就达到了限制，即使你在这些 20 个请求中没有发满 200K 个 Token 。



## 默认速率限制
速率限制与当前账户用量级别、所使用的模型类别（免费或收费）和具体的模型参数大小相关。

### 用量级别与资质
用量级别是指 根据用户月度 API 调用消耗情况来划分的不同等级，针对平台中默认场景：
- 级别设定：当前平台根据用户API消耗金额将速率控制分为6种等级（用量级别0-5）。新用户注册后默认等级为用量级别 0。
- 月度消耗金额选取逻辑：用户当月实际调用 API 推理消耗总金额。
- 级别升级：当账户月度API推理消耗金额达到下一级别标准后，我们会根据您的使用情况自动升级到下一个使用层级。等级升级实时，新等级生效后拥有更高的速率。
- 级别降级：如果您前三个月的实际月度消耗金额都低于当前等级的最低消耗，平台取前三个月实际调用API的最高金额，将您的用量级别降至对应的等级。
- 级别权益：API用量越大，级别越高， RPM 和 TPM 越高。
- 级别查看：您可以在“账户管理-速率限制”中查看您组织当前的用户等级和限制速率。

其他速率限制资源

- [速率限制](https://m09tqret04o.feishu.cn/docx/IAFOd7GW5oJ9hHx57FHcbIPznDh?from=from_copylink)


如果您希望增加速率限制，请访问[速率限制](https://m09tqret04o.feishu.cn/docx/IAFOd7GW5oJ9hHx57FHcbIPznDh?from=from_copylink)页面了解如何提升使用等级。


In [ ]:
# 当前 SiliconCloud 接口整体和 openai 保持一致，可以通过 openai 接口直接使用上述接口。
# 如果是其他框架，也可以直接通过对应语言和框架的类似方案实现，此处不单独举例。
# siliconcloud 其他语言的接口详见：https://docs.siliconflow.cn/reference/chat-completions-3
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [ ]:
import os

# 设置环境变量
os.environ['SILICONCLOUD_API_KEY'] = 'your-api-key in https://cloud.siliconflow.cn/account/ak'

In [ ]:
# 此处使用您的API来替换，对应的 SILICONCLOUD_API_KEY 从[SiliconCloud API密钥](https://cloud.siliconflow.cn/account/ak)页获得。
import openai
import os

# 当前线上版本，总体计划在 8 月 1 日上线
client = openai.OpenAI(
    base_url="https://api.siliconflow.cn/v1",
    api_key=os.environ.get("SILICONCLOUD_API_KEY", "your-api-key in https://cloud.siliconflow.cn/account/ak"))

#  当前公测版本，分层多个用量级别，并针对免费版本提升了 10 倍TPM
pro_client = openai.OpenAI(
    base_url="https://public-beta-api.siliconflow.cn/v1",
    api_key=os.environ.get("SILICONCLOUD_API_KEY", "your-api-key in https://cloud.siliconflow.cn/account/ak"))

## 公测内容

- **基于实际的消费分层限速：**新方案通过RPM、TPM 维度对模型进行限速，用户根据实际API消费金额享有不同限速权益。用的越多，限速越高。
- **免费模型限速提升10倍：**新方案针对免费模型的 RPM 从 100 提升至1000，与此同时引入了 TPM 来避免免费模型被滥用，当前免费模型的 TPM达到了 60000。如果需要使用更高的 TPM 能力，可以进一步使用当前的付费模型。
- **通过“Pro/”区分模型的不同版本：**对于区分部分模型的免费版本和收费版本的模型，并保障兼容性。免费版命名按照原先模型命名规则命名；收费版命名上在模型前边加上 "Pro/"。
- 更多升级内容详见RFC: [SiliconCloud速率限制](https://m09tqret04o.feishu.cn/docx/IAFOd7GW5oJ9hHx57FHcbIPznDh)。


## 公测时间：2024年7月22日至2024年7月31日


## 测试方式：
1. 切换公测域名：
  - 2024年7月22日12:00，新域名生效。请将原先域名 api.siliconflow.cn 替换成公测域名 public-beta-api.siliconflow.cn 进行测试。
2. 进行相关测试：
选择模型进行测试。如果想体验付费小模型的用户，在免费模型名称前加上"Pro/"。允许用户在测试期间体验付费版的服务。
  - ** 付费小模型需要加上"Pro/" 前缀，否则将出现报错**。
  - 请确保在此期间完成所有必要的测试和验证工作。
  - 测试完成后，如果您的业务符合预期，请您切换回原域名api.siliconflow.cn。如果您的测试有相关问题，请联系我们进行反馈。 contact@siliconflow.cn
3. 平台全量上线：
  1. 平台预期在8月1日中午12:00进行全量升级，在api.siliconflow.cn进行全量更新。请您务必在上述时间内进行反馈。
  2. api.siliconflow.cn域名上线后，原先的公测域名 public-beta-api.siliconflow.cn 将于8月4日中午12:00失效。请务必在此之前切换回原域名，以避免服务中断。

  详情查看[SiliconFlow 分层限速方案暨免费模型RPM提升10倍上线通知](https://m09tqret04o.feishu.cn/docx/N5i2dPQg6oFWfyxlGZEclTNOnfd?from=from_copylink)

## 速率限制错误示例

当API请求发送过于频繁时，会出现速率限制错误。如果使用OpenAI Python库，错误信息如下：

```
RateLimitError: Error code: 429 - {'message': 'Request was rejected due to rate limiting. If you want more, please contact contact@siliconflow.cn', 'data': None}
```
以下是触发速率限制错误的示例代码。

In [ ]:
# request a bunch of completions in a loop
for _ in range(110):
    client.chat.completions.create(
        model="Qwen/Qwen2-7B-Instruct",
        messages=[{"role": "user", "content": "你好"}],
        max_tokens=5,
    )

RateLimitError: Error code: 429 - {'message': 'Request was rejected due to rate limiting. If you want more, please contact contact@siliconflow.cn', 'data': None}

目前线上版本的模型在110 个请求时出错，而测试版本的对应模型，在990 个TPM 并发场景下可以正常工作。

当前主要区别：
1. 模型端点：
- 公测版本：https://public-beta-api.siliconflow.cn/v1
- 当前线上：https://api.siliconflow.cn/v1
2. 模型名称：
- 公测版本：Pro/Qwen/Qwen2-7B-Instruct
- 当前线上：Qwen/Qwen2-7B-Instruct

关于上述变化，详情参考：[SiliconCloud速率限制](https://m09tqret04o.feishu.cn/docx/IAFOd7GW5oJ9hHx57FHcbIPznDh) 和 [SiliconFlow 分层限速方案暨免费模型RPM提升10倍上线通知](https://m09tqret04o.feishu.cn/docx/N5i2dPQg6oFWfyxlGZEclTNOnfd?from=from_copylink)

In [ ]:
# 需要注意，此处使用了公测服务的模型端点, 此处验证了这个免费模型超高的TPM能力。
for _ in range(990):
    pro_client.chat.completions.create(
        model="Pro/Qwen/Qwen2-7B-Instruct",
        messages=[{"role": "user", "content": "你好"}],
        max_tokens=5,
    )

KeyboardInterrupt: 

## 如何避免速率限制错误

### 使用指数退避重试

避免速率限制错误的一个简单方法是自动重试请求，并使用随机指数退避。指数退避重试意味着在遇到速率限制错误时，进行短暂休眠，然后重试失败的请求。如果请求仍未成功，休眠时间增加，并重复此过程，直到请求成功或达到最大重试次数。

这种方法有很多好处：

- 自动重试可以避免因速率限制错误导致的崩溃或数据丢失。
- 指数退避使首次重试快速进行，如果前几次重试失败，延迟时间会逐渐增加。
- 随机抖动延迟时间有助于避免所有重试同时发生。

请注意，失败的请求也会计入RPM维度限制，因此不断重新发送请求是无效的。

以下是一些示例解决方案。

#### 示例1: 使用 Tenacity 库

Tenacity 是一个通用重试库，基于Apache 2.0许可，用Python编写，简化了为几乎所有任务添加重试行为的过程。

要为请求添加指数退避，可以使用 tenacity.retry [装饰器](https://peps.python.org/pep-0318/)。下面的示例使用 tenacity.wait_random_exponential 函数为请求添加随机指数退避。

请注意，Tenacity库是第三方工具，SiliconCloud不对其可靠性或安全性做任何保证。

In [ ]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)


completion_with_backoff(model="Qwen/Qwen2-72B-Instruct", messages=[{"role": "user", "content": "以硅基流动是一个好用的平台写一首藏头诗"}])

ChatCompletion(id='0190c5b85febf3bf769a7e54955eaa52', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='以我拙笔绘山河，硅石为基铺绣闼。\n基业稳固驰天下，流光溢彩映星涯。\n动若脱兔驰电子，是处欣欣茂草齐。\n好风凭借力高举，用之不竭智慧源。 \n\n这首藏头诗，每一句的首字顺序排列，即为"以硅基流动是个好用的平台"，表达了对硅基流动平台的肯定和赞美，称颂其在科技领域的稳定、高效以及承载着智慧和未来的无限可能。同时，也强调了硅基流动平台在推动科技进步与发展中的重要性。希望这首藏头诗，能以诗意的形式，展现你所要表达的内容。', role='assistant', function_call=None, tool_calls=None))], created=1721304113, model='Qwen/Qwen2-72B-Instruct', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=151, prompt_tokens=33, total_tokens=184))

#### 示例2: 使用 backoff 库

另一个提供退避和重试功能装饰器的库是 backoff。

与 Tenacity 类似，backoff 库也是第三方工具，OpenAI 对其可靠性或安全性不做任何保证。

In [ ]:
!pip install backoff

In [ ]:
import backoff  # for exponential backoff

@backoff.on_exception(backoff.expo, openai.RateLimitError)
def completions_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)


completions_with_backoff(model="deepseek-ai/DeepSeek-V2-Chat", messages=[{"role": "user", "content": "以硅基流动是一个好用的平台写一首藏头诗"}])


ChatCompletion(id='0190c5b9287d65b64e659e3160c7fffb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='硅林竞技众英才，基石铺就创新台。\n流动能量情激荡，动作敏捷势头来。\n\n平台聚集星辉映，好似银河展奇彩。\n用梦织网连天下，藏龙卧虎展未来。', role='assistant', function_call=None, tool_calls=None))], created=1721304164, model='deepseek-ai/DeepSeek-V2-Chat', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=57, prompt_tokens=16, total_tokens=73))

#### 示例3: 手动实现退避

如果您不想使用第三方库，也可以自己实现退避逻辑。

In [ ]:
# imports
import random
import time

# define a retry decorator
def retry_with_exponential_backoff(
    func,
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
    errors: tuple = (openai.RateLimitError,),
):
    """Retry a function with exponential backoff."""

    def wrapper(*args, **kwargs):
        # Initialize variables
        num_retries = 0
        delay = initial_delay

        # Loop until a successful response or max_retries is hit or an exception is raised
        while True:
            try:
                return func(*args, **kwargs)

            # Retry on specified errors
            except errors as e:
                # Increment retries
                num_retries += 1

                # Check if max retries has been reached
                if num_retries > max_retries:
                    raise Exception(
                        f"Maximum number of retries ({max_retries}) exceeded."
                    )

                # Increment the delay
                delay *= exponential_base * (1 + jitter * random.random())

                # Sleep for the delay
                time.sleep(delay)

            # Raise exceptions for any errors not specified
            except Exception as e:
                raise e

    return wrapper


@retry_with_exponential_backoff
def completions_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)


completions_with_backoff(model="THUDM/glm-4-9b-chat", messages=[{"role": "user", "content": "以硅基流动是一个好用的平台写一首藏头诗"}])

ChatCompletion(id='0190c5d2f0f2ad40192a6a9f20c1f395', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='\n\n硅韵飘香梦犹幻，\n基奠理想筑高墙。\n流年岁月悠悠过，\n动心不已乐章扬。', role='assistant', function_call=None, tool_calls=None))], created=1721305854, model='THUDM/glm-4-9b-chat', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=30, prompt_tokens=18, total_tokens=48))

## 如何在速率限制下最大化批处理吞吐量


如果您处理的是来自用户的实时请求，退避和重试是减少延迟并避免速率限制错误的有效策略。

然而，如果您处理的是大量批量数据，吞吐量比延迟更重要，那么除了退避和重试，还有其他方法可以采用。

### 主动在请求之间添加延迟


如果您不断触发速率限制，然后退避，再次触发，这样会浪费大量请求预算，限制处理吞吐量。

一个潜在的解决方案是计算您的速率限制，并添加相应的延迟（例如，如果速率限制为每分钟20个请求，每个请求添加3-6秒的延迟）。这有助于您在接近速率限制上限的情况下操作，而不会触发限制。

#### 为请求添加延迟的示例

In [ ]:
# imports
import time

# Define a function that adds a delay to a Completion API call
def delayed_completion(delay_in_seconds: float = 1, **kwargs):
    """Delay a completion by a specified amount of time."""

    # Sleep for the delay
    time.sleep(delay_in_seconds)

    # Call the Completion API and return the result
    return pro_client.chat.completions.create(**kwargs)


# Calculate the delay based on your rate limit
rate_limit_per_minute = 1000
delay = 60.0 / rate_limit_per_minute

delayed_completion(
    delay_in_seconds=delay,
    model="Pro/Qwen/Qwen2-7B-Instruct",
    messages=[{"role": "user", "content": "以硅基流动是一个好用的平台写一首藏头诗"}]
)


ChatCompletion(id='0190c5d309684e5abb127379bf42680e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='以梦为马，愿你心飞翔，\n硅基流光，智汇未来航。\n动魄惊心，挑战无极限，\n平台开阔，展翅任翔天。', role='assistant', function_call=None, tool_calls=None))], created=1721305860, model='Pro/Qwen/Qwen2-7B-Instruct', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=38, prompt_tokens=33, total_tokens=71))

## 其他资源：

1. [SiliconCloud速率限制](https://m09tqret04o.feishu.cn/docx/IAFOd7GW5oJ9hHx57FHcbIPznDh?from=from_copylink)
2. [SiliconFlow 分层限速方案暨免费模型RPM提升10倍上线通知](https://m09tqret04o.feishu.cn/docx/N5i2dPQg6oFWfyxlGZEclTNOnfd?from=from_copylink)
3. 如有任何问题，请通过邮箱与我们联系：[contact@siliconflow.cn](mailto:contact@siliconflow.cn)